Here we examine the canonical Hodgkin-Huxley model (Hodgkin and Huxley, 1952). An uncertainty analysis of this model has been performed previously (Valderrama et al., 2015), and we here we repeat a part of that study using Uncertainpy.

We implement the Hodgkin-Huxley model as a Python function

In [1]:
import uncertainpy as un

import numpy as np
from scipy.integrate import odeint


# External stimulus
def I(time):
    return 140 # micro A/cm**2


def valderrama(V_0=-10,
               C_m=1,
               gbar_Na=120,
               gbar_K=36,
               gbar_L=0.3,
               E_Na=112,
               E_K=-12,
               E_l=10.613,
               m_0=0.0011,
               n_0=0.0003,
               h_0=0.9998):

    # Setup time
    end_time = 15          # ms
    dt = 0.025             # ms
    time = np.arange(0, end_time + dt, dt)

    # K channel
    def alpha_n(V):
        return 0.01*(10 - V)/(np.exp((10 - V)/10.) - 1)


    def beta_n(V):
        return 0.125*np.exp(-V/80.)

    def n_f(n, V):
        return alpha_n(V)*(1 - n) - beta_n(V)*n

    def n_inf(V):
        return alpha_n(V)/(alpha_n(V) + beta_n(V))


    # Na channel (activating)
    def alpha_m(V):
        return 0.1*(25 - V)/(np.exp((25 - V)/10.) - 1)

    def beta_m(V):
        return 4*np.exp(-V/18.)

    def m_f(m, V):
        return alpha_m(V)*(1 - m) - beta_m(V)*m

    def m_inf(V):
        return alpha_m(V)/(alpha_m(V) + beta_m(V))


    # Na channel (inactivating)
    def alpha_h(V):
        return 0.07*np.exp(-V/20.)

    def beta_h(V):
        return 1/(np.exp((30 - V)/10.) + 1)

    def h_f(h, V):
        return alpha_h(V)*(1 - h) - beta_h(V)*h

    def h_inf(V):
        return alpha_h(V)/(alpha_h(V) + beta_h(V))


    def dXdt(X, t):
        V, h, m, n = X

        g_Na = gbar_Na*(m**3)*h
        g_K = gbar_K*(n**4)
        g_l = gbar_L

        dmdt = m_f(m, V)
        dhdt = h_f(h, V)
        dndt = n_f(n, V)

        dVdt = (I(t) - g_Na*(V - E_Na) - g_K*(V - E_K) - g_l*(V - E_l))/C_m

        return [dVdt, dhdt, dmdt, dndt]


    initial_conditions = [V_0, h_0, m_0, n_0]

    X = odeint(dXdt, initial_conditions, time)
    values = X[:, 0]

    # Only return from 5 seconds onwards, as in the Valderrama paper
    values = values[time > 5]
    time = time[time > 5]

    # Add info needed by certain spiking features and efel features
    info = {"stimulus_start": time[0], "stimulus_end": time[-1]}

    return time, values, info

We first initialize our model

In [2]:
model = un.Model(run=valderrama,
                 labels=["Time (ms)", "Membrane potential (mV)"])

Then we create the set of parameters

In [3]:
# Define a parameter dictionary
parameters = {"V_0": -10,
              "C_m": 1,
              "gbar_Na": 120,
              "gbar_K": 36,
              "gbar_L": 0.3,
              "m_0": 0.0011,
              "n_0": 0.0003,
              "h_0": 0.9998,
              "E_Na": 112,
              "E_K": -12,
              "E_l": 10.613}

# Create the parameters
parameters = un.Parameters(parameters)

In [4]:
# Set all parameters to have a uniform distribution
# within a 20% (+- 10%) interval around their fixed value
parameters.set_all_distributions(un.uniform(0.2))

Polynomial chaos expansions are recommended as long as the number of uncertain parameters is small (typically >20
), as polynomial chaos expansions in these cases are much faster than quasi-Monte Carlo methods.

> Attention: this is not true for this case, as the model barely runs

In [8]:
# Perform the uncertainty quantification
UQ = un.UncertaintyQuantification(model,
                                  parameters=parameters)
# We set the seed to easier be able to reproduce the result
data = UQ.quantify(seed=10, method='mc')

/Users/teo/.pyenv/versions/compstat/lib/python3.9/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (5000) is equal to `2^n`.
        
  warnings.warn(msg)
Running model: 100%|█████████████████████| 65000/65000 [01:48<00:00, 599.06it/s]


Saving data as: data/valderrama.h5


In [32]:
data = un.Data()
data.load("data/valderrama.h5")

In [33]:
data.data_information

['uncertain_parameters',
 'model_name',
 'incomplete',
 'method',
 'version',
 'seed',
 'model_ignore',
 'error']

In [34]:
data.uncertain_parameters

['V_0',
 'C_m',
 'gbar_Na',
 'gbar_K',
 'gbar_L',
 'm_0',
 'n_0',
 'h_0',
 'E_Na',
 'E_K',
 'E_l']

In [35]:
data["V_0"].variance

KeyError: 'V_0'